<a href="https://colab.research.google.com/github/takky0330/NLP/blob/master/BERTopic_seven.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bertopic

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp /content/drive/MyDrive/BERTopic/00_FAリスト.csv ./00_FAリスト.csv

In [ ]:
import pandas as pd
docs_list = []
csv_pd = pd.read_csv(filepath_or_buffer="./00_FAリスト.csv", encoding="cp932", sep=",")
lists_pd = csv_pd[csv_pd.columns[1]]
lists = list(lists_pd.values)
for lst in lists:
    docs_list = docs_list + lst.rstrip('。').split("。")
    

docs_pd = pd.DataFrame(docs_list, columns=[csv_pd.columns[1]])
docs = docs_pd[csv_pd.columns[1]].tolist()

In [ ]:
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from sklearn.cluster import KMeans
cluster_model = KMeans(n_clusters=20)
#from sklearn.mixture import GaussianMixture
#cluster_model = GaussianMixture(n_components=3)

sentence_model = SentenceTransformer("sonoisa/t5-base-japanese")
#topic_model = BERTopic(embedding_model=sentence_model, calculate_probabilities=True, verbose=True, nr_topics="20")
topic_model = BERTopic(embedding_model=sentence_model, calculate_probabilities=True, verbose=True)
#topic_model = BERTopic(embedding_model=sentence_model, hdbscan_model=cluster_model, calculate_probabilities=True, verbose=True)
topics, probs = topic_model.fit_transform(docs)

In [ ]:
topic_model.get_topic_info()

In [ ]:
topic_model.get_topic(0)

In [ ]:
topics_pd = pd.DataFrame(topics, index=docs_pd.index, columns=['topics'])
topics_pd = pd.concat([topics_pd, docs_pd], axis=1)
topics_pd

In [ ]:
topics_pd[topics_pd['topics'] == 0]

In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_distribution(probs[200], min_probability=0.0)

In [ ]:
topic_model.visualize_hierarchy()

In [ ]:
topic_model.visualize_barchart(top_n_topics=20)

In [ ]:
topics_pd.to_csv('./BERTopic_sonoisa.csv', sep=',', encoding='cp932')
!cp ./BERTopic_sonoisa.csv /content/drive/MyDrive/BERTopic/